## Plot the throughput of experiment 1 of version 3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io
import glob

## Function Read CSV files of Throughput from Iperf log


In [ ]:
def getDataframeThru(df,start_row,measurement_interval,header_range):
    '''
    This functions will import the data from txt file and return the dataframe without the header of txt file.
    Input: 
        measurement_interval = 30 (sec) : 
        header_range = 10 lines
        start_row = 0 
    Return: 
        df1t : dataframe of througput and jitter
    '''
    
    df1 = df.drop(labels=range(start_row, header_range), axis=0)
    df1t = df1.drop(labels=range(measurement_interval, len(df)), axis=0)
    
    return df1t

In [ ]:
def getDatafromTxT(filename, headerrange):
    """
    Get dataframe from txt file:
    filename : xxx.txt
    headerrange : number of lines that needed to be removed. 
    
    return : df : datafame type
    """
    h = headerrange + 1
    skip_1 = list(range(0,h, 1))
    df = pd.read_csv(filename,
        skiprows=skip_1,
        header=None,
        delimiter=' ',
        skipinitialspace=True,
        error_bad_lines=False)
    return df


In [ ]:
## Find start row index of itteration
def getStartEndID(df,start_data,end_data):
    """
    to clean dataframe and return the data with new header
    Input: 
        df : datafram without header of txt file 
    Output
        strat_indices_list : start indices list
    """
    # creating and passing series to new column
    df["Start"]= df[2].str.find(start_data)
    df["End"]= df[2].str.find(end_data)

    index = df.index
    strat_indices = index[df["Start"]==0.0]
    strat_indices_list = strat_indices.tolist()
    end_indices = index[df["End"]==0.0]
    end_indices_list = end_indices.tolist()
    
    
    return strat_indices_list, end_indices_list
    

In [ ]:
def getCleanData(df,strat_indices_list,end_indices_list):
    """
    
    """
    df_all = df.drop(labels=range(1, len(df)), axis=0) # create new df
    start_row = 0
    c = 0
    for i in strat_indices_list: 
        
        h = i
        print('h =',h)
        m = end_indices_list[c]
        print('m =', m)
        df1 = getDataframeThru(df,start_row,m,h)
        print('df1 = ', df1)
        result = pd.concat([df_all,df1])
        df_all = result
        c = c + 1
        if i == 0:
            
            df_all = df_all.drop(labels=0, axis=0)
        
        
    return df_all

In [ ]:
def superClean(filename,headerrange,start_data,end_data):
    """
    Clean Data from CSV file with remove the unnecessary header
    """
    df = getDatafromTxT(filename, headerrange)
    strat_indices_list, end_indices_list = getStartEndID(df,start_data,end_data)
    df_all = getCleanData(df,strat_indices_list,end_indices_list)
    df_all_new = df_all.drop(df_all.columns[[0,1,3,5,7,9]], axis=1)  # Replace new columns header
    df_all_new.rename({2 :'Interval', 4 : 'Transfer', 6 :'Bitrate', 8 :'Jitter', 10 :'Lost/Total Datagrams'}, axis=1, inplace=True)
    df = df_all_new.drop(range(0,1))
    df_all_new['Bitrate'] = df['Bitrate'].astype(float) 
    time = np.array(range(len(df_all_new.index)))

    df_all_new['Time'] = time
    df_all_new['Time'] = df_all_new['Time'].astype(int) 
    # avergae throughput 
    sumThroughput = df_all_new['Bitrate'].sum()
    avgSumThroughput = sumThroughput/len(time)
    print('avg  :', avgSumThroughput)
    var_throughput = df_all_new['Bitrate'].var()
    print('var  :', var_throughput)
    
    return avgSumThroughput, var_throughput

    
    

In [ ]:
def readCSV2pd_Thru(directoryPath,tf_load,edge_name,start_data,end_data,headerrange):
    """
    This function is to read a CSV file and return the average value and varience
    input: directoryPath : path of file names
            tf_load : list of traffic load
    """
    avg_Thr = []
    var_Thr = []
    for tf in tf_load:
        cpu_data = pd.DataFrame()
        for file_name in glob.glob(directoryPath+edge_name+str(tf)+'.csv'):
            avg_thr,var_thr = superClean(file_name,headerrange,start_data,end_data)
            
            avg_Thr.append(avg_thr)
            var_Thr.append(var_thr)
    return avg_Thr, var_Thr

## Read File CSV

In [ ]:
headerrange = 7
start_data = '9.0-10.0'
end_data = '60.0-61.0'
tf_load  = [1,i*2 for i in range(4,20)]
edge_name = 'edge1_M'
directoryPath = '/Users/kalika/PycharmProjects/Privacy_SDN_Edge_IoT/PlanB/CPU_utilization_Experiment/version3_Experiment_style/Experiment1/Edge1_iperf_log/'
avg_thr, var_thr = readCSV2pd_Thru(directoryPath,tf_load,edge_name,start_data,end_data,headerrange)

In [ ]:
print('avg',avg_thr)
print('var',var_thr)

In [ ]:
headerrange = 7
start_data = '9.0-10.0'
end_data = '60.0-61.0'
tf_load  = [i*2 for i in range(4,20)]
edge_name = 'edge2_M'
directoryPath = '/Users/kalika/PycharmProjects/Privacy_SDN_Edge_IoT/PlanB/CPU_utilization_Experiment/version3_Experiment_style/Experiment1/Edge2_iperf_log/'
avg_thr2, var_thr2 = readCSV2pd_Thru(directoryPath,tf_load,edge_name,start_data,end_data,headerrange)

In [ ]:
print('avg',avg_thr2)
print('var',var_thr2)

## Plot Throughput

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(1, 1, 1)


ax.plot(tf_load, avg_thr, color='green', linestyle='dashed', linewidth = 2,
         marker='o', markerfacecolor='green', markersize=10,label="Edge 1")
ax.plot(tf_load, avg_thr2, color='red', linestyle='dashed', linewidth = 2,
         marker='x', markerfacecolor='red', markersize=10,label="Edge 2")
plt.ylim(0,30)
plt.xlim(0,40)
plt.xlabel('Traffic load $\lambda_{1,2}$ (Mbps)')
# naming the y axis
plt.ylabel('Average of CPU utilization (%)')

plt.legend()

plt.show()